Start


In [202]:
import requests
from requests.auth import HTTPBasicAuth

BASE_URL = "http://localhost:2480"
AUTH = HTTPBasicAuth("root", "playwithdata")

Create a database

In [203]:
def create_database(name):
    url = f"{BASE_URL}/api/v1/server"
    payload = { "command": f"create database {name}" }
    response = requests.post(url, json=payload, auth=AUTH)
    print(f"[CREATE DB] Status: {response.status_code}\nResponse: {response.text}\n")

create_database("test_db")

[CREATE DB] Status: 200
Response: {"result":"ok"}



In [204]:
def run_sql(command):
    url = f"{BASE_URL}/api/v1/command/testing"
    payload = { "language": "sql", "command": command }
    response = requests.post(url, json=payload, headers=HEADERS, auth=AUTH)
    print(f"> {command}\nStatus: {response.status_code}\nResponse:\n{response.text}\n")
    

In [28]:
run_sql("create vertex type Course")
run_sql("create edge type EnrolledIn")

> create vertex type Course
Status: 200
Response:
{"result":[{"operation":"create vertex type","typeName":"Course"}]}

> create edge type EnrolledIn
Status: 200
Response:
{"result":[{"operation":"create edge type","typeName":"EnrolledIn"}]}



In [29]:
run_sql("insert into Student set name = 'Vamsi', age = 23")
run_sql("insert into Course set title = 'Data Science'")

> insert into Student set name = 'Vamsi', age = 23
Status: 200
Response:
{"result":[{"@rid":"#1:0","@type":"Student","@cat":"v","name":"Vamsi","age":23}]}

> insert into Course set title = 'Data Science'
Status: 200
Response:
{"result":[{"@rid":"#4:0","@type":"Course","@cat":"v","title":"Data Science"}]}



In [30]:
# Step 3: Create relationship
run_sql("create edge EnrolledIn from (select from Student where name='Vamsi') to (select from Course where title='Data Science')")

> create edge EnrolledIn from (select from Student where name='Vamsi') to (select from Course where title='Data Science')
Status: 200
Response:
{"result":[{"@rid":"#7:0","@type":"EnrolledIn","@cat":"e","@in":"#4:0","@out":"#1:0"}]}



In [33]:
commands = [
    "insert into Student set name='uday', age=24",
    "insert into Course set title='MTS'",
    "create edge EnrolledIn from (select from Student where name='uday') to (select from Course where title='MTS')"
]

for cmd in commands:
    run_sql(cmd)


> insert into Student set name='uday', age=24
Status: 200
Response:
{"result":[{"@rid":"#1:1","@type":"Student","@cat":"v","name":"uday","age":24}]}

> insert into Course set title='MTS'
Status: 200
Response:
{"result":[{"@rid":"#4:1","@type":"Course","@cat":"v","title":"MTS"}]}

> create edge EnrolledIn from (select from Student where name='uday') to (select from Course where title='MTS')
Status: 200
Response:
{"result":[{"@rid":"#7:1","@type":"EnrolledIn","@cat":"e","@in":"#4:1","@out":"#1:1"}]}



In [34]:
run_sql("create edge type friends" )

> create edge type friends
Status: 200
Response:
{"result":[{"operation":"create edge type","typeName":"friends"}]}



In [95]:
run_sql("create edge friends from (select from Student where name = 'uday') to (select from Student where name = 'Vamsi')")

> create edge friends from (select from Student where name = 'uday') to (select from Student where name = 'Vamsi')
Status: 200
Response:
{"result":[{"@rid":"#8:5","@type":"friends","@cat":"e","@in":"#1:0","@out":"#1:1"}]}



<Response [200]>

In [200]:
import urllib.parse

In [145]:
run_sql("SELECT expand(out('EnrolledIn')) FROM Student WHERE name = 'Vamsi'")

> SELECT expand(out('EnrolledIn')) FROM Student WHERE name = 'Vamsi'
Status: 200
Response:
{"result":[{"@rid":"#4:0","@type":"Course","@cat":"v","title":"Data Science"}]}



In [173]:
run_sql("SELECT expand(out('friends')) FROM Student WHERE name = 'Vamsi'")

> SELECT expand(out('friends')) FROM Student WHERE name = 'Vamsi'
Status: 200
Response:
{"result":[{"@rid":"#1:2","@type":"Student","@cat":"v","name":"Yash"},{"@rid":"#1:1","@type":"Student","@cat":"v","name":"uday","age":24}]}



In [201]:
def get_data(sql_query):
    encoded_query = urllib.parse.quote(sql_query)
    url = f"{BASE_URL}/api/v1/query/testing/sqlscript/{encoded_query}"
    response = requests.get(url, auth=AUTH)
    #print(f"> {sql_query}\nStatus: {response.status_code}\nResponse:\n{response.json() if response.text else 'No Response'}\n")
    return response.json()['result']
# Usage
all_students = get_data("SELECT FROM Student")
vamsi_courses = get_data("SELECT expand(out('EnrolledIn')) FROM Student WHERE name = 'Vamsi'")
vamsi_friends = get_data("SELECT expand(out('friends')) FROM Student WHERE name = 'Vamsi'") 

ConnectionError: HTTPConnectionPool(host='localhost', port=2480): Max retries exceeded with url: /api/v1/query/testing/sqlscript/SELECT%20FROM%20Student (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002420D517890>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [169]:
all_students

[{'@rid': '#1:0',
  '@type': 'Student',
  '@cat': 'v',
  'name': 'Vamsi',
  'age': 23,
  'email': 'vamsi@example.com',
  'address': 'Hyderabad'},
 {'@rid': '#1:1', '@type': 'Student', '@cat': 'v', 'name': 'uday', 'age': 24},
 {'@rid': '#1:2', '@type': 'Student', '@cat': 'v', 'name': 'Yash'},
 {'@rid': '#1:5', '@type': 'Student', '@cat': 'v', 'name': 'akhil'},
 {'@rid': '#1:6', '@type': 'Student', '@cat': 'v', 'name': 'Yash'},
 {'@rid': '#1:9', '@type': 'Student', '@cat': 'v', 'name': 'Yash'}]

In [170]:
vamsi_courses

[{'@rid': '#4:0', '@type': 'Course', '@cat': 'v', 'title': 'Data Science'}]

In [171]:
vamsi_friends

[{'@rid': '#1:2', '@type': 'Student', '@cat': 'v', 'name': 'Yash'},
 {'@rid': '#1:1', '@type': 'Student', '@cat': 'v', 'name': 'uday', 'age': 24}]

In [155]:
run_sql("create property Student.email string")
run_sql("create property Student.address string")
run_sql("update Student set email='vamsi@example.com', address='Hyderabad' where name='Vamsi'")

> create property Student.email string
Status: 500
Response:
{"error":"Error on transaction commit","detail":"Property \u0027Student.email\u0027 already exists","exception":"com.arcadedb.exception.CommandExecutionException"}

> create property Student.address string
Status: 500
Response:
{"error":"Error on transaction commit","detail":"Property \u0027Student.address\u0027 already exists","exception":"com.arcadedb.exception.CommandExecutionException"}

> update Student set email='vamsi@example.com', address='Hyderabad' where name='Vamsi'
Status: 200
Response:
{"result":[{"count":1}]}



In [182]:
def run_sql_script(script):
    url = f"{BASE_URL}/api/v1/command/testing"
    payload = { "language" : 'sqlscript',"command": script }
    response = requests.post(url, json=payload, headers=HEADERS, auth=AUTH)
    print(f"[RUN SCRIPT] Status: {response.status_code}\nResponse:\n{response.text}\n")


In [179]:
add_friends = """create vertex Student set name = 'Yash';
create vertex Student set name = 'Kay';
create vertex Student set name = 'Jay';"""
run_sql_script(add_friends)

In [189]:
vamsi_friends = """
let Yash = (select from Student where name = 'Yash');
let others = (select from Student where name != 'Yash');

foreach (record in $others) {
  create edge friends from $Yash[0] to $record;
  create edge friends from $record to $Yash[0];
}


"""
run_sql_script(vamsi_friends)

[RUN SCRIPT] Status: 200
Response:
{"result":[]}



Query using rids

In [80]:

# Step 1: Get Vamsi RID
vamsi_query = "select from Student where name = 'Vamsi'"
vamsi_res = run_sql(vamsi_query).json()
vamsi_rid = vamsi_res['result'][0]['@rid']

print(vamsi_rid)
# Step 2: Get other students' RIDs
others_query = "select from Student where name != 'Vamsi'"
others_res = run_sql(others_query).json()
other_rids = [record['@rid'] for record in others_res['result']]
print(other_rids)
# Step 3: Create mutual FRIEND edges
for other_rid in other_rids:
    create1 = f"create edge friends from {vamsi_rid} to {other_rid}"
    run_sql(create1)

print("✅ FRIEND edges successfully created between Vamsi and others.")

#1:0
['#1:1', '#1:2', '#1:3', '#1:4']
✅ FRIEND edges successfully created between Vamsi and others.


In [89]:
# 1. Create the Course vertices
create_courses_script = """
create vertex Course set title = 'Biology';
create vertex Course set title = 'History';
"""
run_sql_script(create_courses_script)

# 2. Get RIDs of students and courses
entities = {
    "Jay": "Student",
    "Kay": "Student",
    "Biology": "Course",
    "History": "Course"
}

name_to_rid = {}
for name, class_name in entities.items():
    query = f"select from {class_name} where {'title' if class_name == 'Course' else 'name'} = '{name}'"
    result = get_data(query)
    if result:
        name_to_rid[name] = result[0]["@rid"]
    else:
        print(f"⚠️ Could not find {name} in {class_name}")

# 3. Create ENROLLED_IN edges
enrollment_script = """
create edge ENROLLED_IN if not exists;
"""

# Jay → Biology
enrollment_script += f"create edge ENROLLED_IN from {name_to_rid['Jay']} to {name_to_rid['Biology']};\n"
# Kay → History
enrollment_script += f"create edge ENROLLED_IN from {name_to_rid['Kay']} to {name_to_rid['History']};\n"

# 4. Run the enrollment script
run_sql_script(enrollment_script)


[RUN SCRIPT] Status: 200
Response:
{"result":[{"@rid":"#4:9","@type":"Course","@cat":"v","title":"History"}]}

> select from Student where name = 'Jay'
Status: 200
Response:
{'result': [{'@rid': '#1:4', '@type': 'Student', '@cat': 'v', 'name': 'Jay'}]}

⚠️ Could not find Jay in Student
> select from Student where name = 'Kay'
Status: 200
Response:
{'result': [{'@rid': '#1:3', '@type': 'Student', '@cat': 'v', 'name': 'Kay'}]}

⚠️ Could not find Kay in Student
> select from Course where title = 'Biology'
Status: 200
Response:
{'result': [{'@rid': '#4:2', '@type': 'Course', '@cat': 'v', 'title': 'Biology'}, {'@rid': '#4:4', '@type': 'Course', '@cat': 'v', 'title': 'Biology'}, {'@rid': '#4:6', '@type': 'Course', '@cat': 'v', 'title': 'Biology'}, {'@rid': '#4:8', '@type': 'Course', '@cat': 'v', 'title': 'Biology'}]}

⚠️ Could not find Biology in Course
> select from Course where title = 'History'
Status: 200
Response:
{'result': [{'@rid': '#4:3', '@type': 'Course', '@cat': 'v', 'title': 'Hi

KeyError: 'Jay'

In [100]:
delete_history_script = "delete vertex Course where title = 'History'"
run_sql(delete_history_script)

> delete vertex Course where title = 'History'
Status: 500
Response:
{"error":"Error on transaction commit","detail":"delete vertex Course where title \u003d \u0027History\u0027","exception":"com.arcadedb.exception.CommandSQLParsingException"}



<Response [500]>

In [101]:
get_data("select from Course where title = 'History'")

> select from Course where title = 'History'
Status: 200
Response:
{'result': [{'@rid': '#4:3', '@type': 'Course', '@cat': 'v', 'title': 'History'}, {'@rid': '#4:5', '@type': 'Course', '@cat': 'v', 'title': 'History'}, {'@rid': '#4:7', '@type': 'Course', '@cat': 'v', 'title': 'History'}, {'@rid': '#4:9', '@type': 'Course', '@cat': 'v', 'title': 'History'}]}



In [198]:
# Step 1: Get the RID of the course
result = get_data("select from Course where title = 'History'")
if result:
    rid = result[0]["@rid"]
    # Step 2: Delete using RID
    run_sql(f"delete vertex {rid}")
else:
    print("Course 'History' not found.")


ConnectionError: HTTPConnectionPool(host='localhost', port=2480): Max retries exceeded with url: /api/v1/query/testing/sqlscript/select%20from%20Course%20where%20title%20%3D%20%27History%27 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002420D447A10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

Delete a vertex or a edge

In [160]:
run_sql("delete from Student where name = 'Jay'")

> delete from Student where name = 'Jay'
Status: 200
Response:
{"result":[{"count":3}]}



In [163]:
run_sql("delete from Student where name = 'Kay'")

> delete from Student where name = 'Kay'
Status: 200
Response:
{"result":[{"count":3}]}



Checking for the existing vertices 

In [190]:
script = """
let student = select from Student where name = 'Rathnala';
if ($student.size() == 0) {
  insert into Student set name = 'Rathnala';
}
"""
run_sql_script(script)

[RUN SCRIPT] Status: 200
Response:
{"result":[{"@rid":"#1:18","@type":"Student","@cat":"v","name":"Rathnala"}]}



If one of the vertices exists, create the other and then add the edge.

If none exist, create both and then add the edge.

If both exist, just add the edge.

In [194]:

script = """
-- Step 1: Create types if they don't exist
create vertex type Student if not exists;
create edge type FRIEND if not exists;

-- Step 2: Ensure both vertices exist
let vamsi = select from Student where name = 'Vamsi';
if ($vamsi.size() = 0) {
  let vamsi = insert into Student set name = 'Vamsi';
}

let sandeep = select from Student where name = 'Sandeep';
if ($sandeep.size() = 0) {
  let sandeep = insert into Student set name = 'Sandeep';
}

-- Step 3: Create edge if it doesn't exist
let edgeExists = select from FRIEND where out.name = 'Vamsi' and in.name = 'Sandeep';
if ($edgeExists.size() = 0) {
  create edge FRIEND from (select from Student where name = 'Vamsi') to (select from Student where name = 'Sandeep');
}

"""
run_sql_script(script)

[RUN SCRIPT] Status: 200
Response:
{"result":[{"@rid":"#15:0","@type":"FRIEND","@cat":"e","@in":"#1:19","@out":"#1:0"}]}



In [195]:
sql_query = "SELECT out('FRIEND') FROM Student WHERE name = 'Vamsi'"
get_data(sql_query)


ConnectionError: HTTPConnectionPool(host='localhost', port=2480): Max retries exceeded with url: /api/v1/query/testing/sqlscript/SELECT%20out%28%27FRIEND%27%29%20FROM%20Student%20WHERE%20name%20%3D%20%27Vamsi%27 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002420C264D90>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))